## Mongo

In [ ]:
# brew services start mongodb-community@6.0
# on command line

In [ ]:
import pymongo as pm
import pprint as pp
import time

In [ ]:
client = pm.MongoClient()
db = client.ronin
client, db

In [ ]:
db.axiesales

## Lunacian Summer

**Axie Listing**
Approve https://explorer.roninchain.com/tx/0xa91b254adea5cf7f2c1aff8b088d9224f7865871ea532135546acd626ca0b654

**Market Gateway**
0xfff9ce5f71ca6178d3beecedb61e7eff1602950e
https://explorer.roninchain.com/address/ronin:fff9ce5f71ca6178d3beecedb61e7eff1602950e

**Axie Purchase**
ORDER_EXCHANGE on Market Gateway https://explorer.roninchain.com/tx/0x48478beca5730d76d4e126b9dbff4493f7e7199c9ccad07b96b32b571570855d

Great Reset https://axie.substack.com/p/greatreset (since Aug 5, 2022)

**Community "Treasury"**
https://explorer.roninchain.com/address/ronin:a99cacd1427f493a95b585a5c7989a08c86a616b

## Thoughts

For the Summer Event, were people doing Step 1 (breeding axies without summer parts for axies summer parts) or Step 1+ (breeding axies with summer parts for summer parts)?
    
Then, look at social data on Twitter.
* what did people post? the rares? or first times?
    * compare events
    
Problem statement: Gaming companies cannot do the same event over and over. The Lunacian Express event was a flop. The Lunacian Summer event was a success. But they cannot do breeding events all the time because users would lose interest. Thus, they must find the underlying principle to the success of the Lunacian Summer event and apply that to a variety of events. They could do another "Express" event but with the correct mechanisms to encourage token burn. Or they could do *any* event with the correct mechanisms.

Both Lunacian Express and Lunacian Summer events were both for cosmetic items.

Was it because they could use the axies in battle? Check battle logs.

Or did they just want to collect the summer axies?

Success can be measured by:
- event usage
    - activity as function of time, progress on the event
    - what is the point of user?
- marketplace tx
    - over time?
    - individual decisions
        - which ones (1/6 or n/6)?
        - do they want to produce it themselves or buy it?
    - what process are people joining or exiting?
- value burned
    - tokens and NFTs
- social metrics
- "consulting" solutions
    - gotcha roll, crafting
    
- https://web3py.readthedocs.io/en/latest/contracts.html?highlight=decode#methods
- https://docs.google.com/spreadsheets/d/1Eh4453zDYuo814ZOkyfl1lDjwwIhZpJH39lzQFG2YS4/edit#gid=1278785847

### web3py

In [ ]:
from web3 import Web3, contract
import json, requests

In [ ]:
USER_AGENT = "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.77 Safari/537.36"
headers = {
  "Content-Type": "application/json",
  "User-Agent": USER_AGENT }

web3 = Web3(Web3.HTTPProvider('https://api.roninchain.com/rpc', request_kwargs={ "headers": headers }))

In [ ]:
contract_address_axie = Web3.toChecksumAddress('0x32950db2a7164ae833121501c797d79e7b79d74c')
contract_address_axs = Web3.toChecksumAddress('0x97a9107c1793bc407d6f527b77e7fff4d812bece')
contract_address_slp = Web3.toChecksumAddress('0xa8754b9fa15fc18bb59458815510e40a12cd2014')
contract_address_axie, contract_address_axs, contract_address_slp

In [ ]:
tx = web3.eth.get_transaction('0x7c5042ea44e6e78128a09f3a8b686267046b2013a7c5653e9fabbd40242f3c03')
tx

In [ ]:
receipt = web3.eth.get_transaction_receipt('0x7c5042ea44e6e78128a09f3a8b686267046b2013a7c5653e9fabbd40242f3c03')
receipt

In [ ]:
receipt.logs[0]

### Marketplace GraphQL

In [ ]:
USER_AGENT = "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.77 Safari/537.36"
headers = {
  "Content-Type": "application/json",
  "User-Agent": USER_AGENT }
url = 'https://graphql-gateway.axieinfinity.com/graphql'

In [ ]:
null = None
auctionType = ["All", "Sale"]
body = {
    "operationName": "GetAxieBriefList",
    "variables": {
        "from":0, "size":2,
        "sort":"PriceAsc", "auctionType":auctionType[0], "owner":null,
        "criteria": {
            "region":null, "parts":null, "bodyShapes":null, "classes":null, "stages":null,
            "numMystic":null,"pureness":null,"title":null,"breedable":null,"breedCount":null,
            "hp":[],"skill":[],"speed":[],"morale":[],"purity":[],"numJapan":null,
            "numXmas":null,"numShiny":null,"numSummer":[1],
            "ppBeast":null,"ppAquatic":null,"ppPlant":null,"ppBug":null,
            "ppBird":null,"ppReptile":null,"ppMech":null,"ppDawn":null,"ppDusk":null
        }
    },
    "query": "query GetAxieBriefList($auctionType: AuctionType, $criteria: AxieSearchCriteria, $from: Int, $sort: SortBy, $size: Int, $owner: String) {\n axies(\n auctionType: $auctionType\n criteria: $criteria\n from: $from\n sort: $sort\n size: $size\n owner: $owner\n ) {\n total\n results {\n ...AxieBrief\n __typename\n }\n __typename\n }\n}\n\nfragment AxieBrief on Axie {\n id\n name\n stage\n class\n breedCount\n image\n title\n genes\n newGenes\n battleInfo {\n banned\n __typename\n }\n order {\n id\n currentPrice\n currentPriceUsd\n __typename\n }\n parts {\n id\n name\n class\n type\n specialGenes\n __typename\n }\n __typename\n}\n"
}
response = requests.post(url=url, headers=headers, json=body)
response, response.content

In [ ]:
data = json.loads(response.content)
list(data.keys()), list(data['data'].keys()), list(data['data']['axies'].keys())

In [ ]:
total = data['data']['axies']['total']
axies = data['data']['axies']['results']
total, axies[0]

### Create MongoDB Collection

In [ ]:
db.list_collection_names()

In [ ]:
db['summeraxies']

In [ ]:
USER_AGENT = "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.77 Safari/537.36"
headers = {
  "Content-Type": "application/json",
  "User-Agent": USER_AGENT }
url = 'https://graphql-gateway.axieinfinity.com/graphql'
null = None
auction_types = ["All", "Sale"]
axie_classes = ['Beast', 'Aquatic', 'Plant', 'Bug', 'Bird', 'Reptile', 'Mech', 'Dawn', 'Dusk']

def fetchSummerAxies(from_index=0, size=25, auction_type="All", num_summer=[1], num_shiny=None, axie_class=None):
    body = {
        "operationName": "GetAxieBriefList",
        "variables": {
            "from":from_index, "size":size,
            "sort":"PriceAsc", "auctionType":auction_type, "owner":None,
            "criteria": {
                "region":None, "parts":None, "bodyShapes":None, "classes":axie_class, "stages":None,
                "numMystic":None, "pureness":None, "title":None, "breedable":None, "breedCount":None,
                "hp":[], "skill":[], "speed":[], "morale":[], "purity":[], "numJapan":None,
                "numXmas":None, "numShiny":num_shiny, "numSummer":num_summer,
                "ppBeast":None, "ppAquatic":None, "ppPlant":None, "ppBug":None,
                "ppBird":None, "ppReptile":None, "ppMech":None, "ppDawn":None, "ppDusk":None
            }
        },
        "query": "query GetAxieBriefList($auctionType: AuctionType, $criteria: AxieSearchCriteria, $from: Int, $sort: SortBy, $size: Int, $owner: String) {\n axies(\n auctionType: $auctionType\n criteria: $criteria\n from: $from\n sort: $sort\n size: $size\n owner: $owner\n ) {\n total\n results {\n ...AxieBrief\n __typename\n }\n __typename\n }\n}\n\nfragment AxieBrief on Axie {\n id\n name\n stage\n class\n breedCount\n image\n title\n genes\n newGenes\n battleInfo {\n banned\n __typename\n }\n order {\n id\n currentPrice\n currentPriceUsd\n __typename\n }\n parts {\n id\n name\n class\n type\n specialGenes\n __typename\n }\n __typename\n}\n"
    }
    response = requests.post(url=url, headers=headers, json=body)
    data = json.loads(response.content)
    total = data['data']['axies']['total']
    axies = data['data']['axies']['results']
    return total, axies

In [ ]:
page_size = 24
for num_summer in [1, 2, 3, 4, 5, 6]:
    for axie_class in axie_classes:
        # graphql has limits to index 10,000
        print(f"Fetching for {num_summer} summer parts for {axie_class}...")
        total, _ = fetchSummerAxies(
            auction_type="All", num_summer=[num_summer], axie_class=[axie_class]
        )
        for i in range(0, total, page_size):
            print(f"\tfrom {i} to {i+page_size} out of {total}...")
            _, axies = fetchSummerAxies(
                from_index=i, size=page_size, 
                auction_type="All", num_summer=[num_summer], axie_class=[axie_class]
            )
            if len(axies)==0:
                break
            axies_annotated = []
            for axie in axies:
                axie['numSummer'] = num_summer
                axie['numShiny'] = 0
                axies_annotated.append(axie)
            db.summeraxies.insert_many(axies_annotated)
            time.sleep(1)
    total_in_ql, _ = fetchSummerAxies(auction_type="All", num_summer=[num_summer])
    total_in_db = db.summeraxies.count_documents({'numSummer': num_summer})
    print(f"Total {total_in_ql} from query. Total {total_in_db} from database.")

In [ ]:
for num_summer in [1, 2, 3, 4, 5, 6]:
    total_in_ql, _ = fetchSummerAxies(auction_type="All", num_summer=[num_summer])
    total_in_db = db.summeraxies.count_documents({'numSummer': num_summer})
    print(f"Total {total_in_ql} from query. Total {total_in_db} from database.")

In [ ]:
# cursor = db.summeraxies.aggregate(
#     [
#         {"$group": {"_id": "$id", "unique_ids": {"$addToSet": "$_id"}, "count": {"$sum": 1}}},
#         {"$match": {"count": { "$gte": 2 }}}
#     ]
# )

# response = []
# for doc in cursor:
#     del doc["unique_ids"][0]
#     for id in doc["unique_ids"]:
#         response.append(id)

# db.summeraxies.delete_many({"_id": {"$in": response}})

In [ ]:
db.summeraxies.create_index('id')#, unique=True)

### Grab data archive

https://get.ronin.rest/export/

In [ ]:
# db.axietransfers.create_index([('axie', pm.DESCENDING), ('from', pm.TEXT)])
db.axietransfers.create_index([('axie', pm.DESCENDING)])

In [ ]:
next(db.axietransfers.find().sort('created_at', pm.DESCENDING).limit(1))

In [ ]:
db.transactions.count_documents({})

In [ ]:
db.transactions.create_index([('block', pm.DESCENDING), ('to', pm.TEXT)])

In [ ]:
next(db.transactions.find().sort('created_at', pm.DESCENDING).limit(1))

### Get axie mint

In [ ]:
summer_axie = db.summeraxies.find_one()
summer_axie['id']

In [ ]:
import pprint as pp

axie_mint = db.axietransfers.find_one(
    {'axie': int(summer_axie['id']), 'from': '0x0000000000000000000000000000000000000000'}
)
axie_mint

In [ ]:
f = open("axie-abi.json", "r")
lines = f.read()
abi_str = json.dumps(json.loads(lines))
contract_axie = web3.eth.contract(address=contract_address_axie, abi=abi_str)

In [ ]:
set([abi['type'] for abi in contract_axie.abi if 'type' in abi])

In [ ]:
function_breed = contract_axie.find_functions_by_name('breedAxies')[0]
function_breed

In [ ]:
contract_address_axie, contract_address_axs, contract_address_slp

In [ ]:
def breed_tx_for_axie(axie_id):
    axie_mint = db.axietransfers.find_one(
        {'axie': axie_id, 'from': '0x0000000000000000000000000000000000000000'}
    )
    if not axie_mint:
        print(f"No mint tx for axie {axie_id} in DB.")
        return
    cursor = db.transactions.find({
        'block': axie_mint['block'],
        'to': contract_address_axie.lower()
    })
    for tx in cursor:
        tx_whole = web3.eth.get_transaction(tx['hash'])
        try:
            tx_input = contract_axie.decode_function_input(tx_whole.input)
        except:
#             print(f"\nUnrecognized function for tx hash {tx['hash']}\n")
            continue
        if str(tx_input[0])==str(function_breed):
            sire_id = tx_input[1]['_sireId']
            matron_id = tx_input[1]['_matronId']
            receipt = web3.eth.get_transaction_receipt(tx['hash'])
            axie_spawn = contract_axie.events.AxieSpawn().processReceipt(receipt)
            if len(axie_spawn) > 0 and axie_id==axie_spawn[0]['args']['_axieId']:
                fee_axs = fee_slp = 0
                for log in receipt.logs:
                    if log.address==contract_address_axs:
                        fee_axs = int(log.data, 16) / 10**18
                    elif log.address==contract_address_slp:
                        fee_slp = int(log.data, 16)
                tx_db = db.transactions.find_one({'hash': tx_whole['hash'].hex()})
                created_at = tx_db['created_at']
                return {
                    'blockNumber': tx_whole['blockNumber'],
                    'createdAt': created_at,
                    'hash': tx_whole['hash'],
                    'from': tx_whole['from'],
                    'gas': tx_whole['gas'],
                    'sireId': sire_id,
                    'matronId': matron_id,
                    'feeAxs': fee_axs,
                    'feeSlp': fee_slp
                }

In [ ]:
breed_tx_for_axie(11354512)

In [ ]:
total_count = db.summeraxies.count_documents({})
cursor = db.summeraxies.find()
for i, summer_axie in enumerate(cursor):
    if 'breedTx' in summer_axie:
        continue
    print(f"\r{i+1}/{total_count}", end='')
    if 'breedTx' in summer_axie:
        continue
    breed_tx = breed_tx_for_axie(int(summer_axie['id']))
    db.summeraxies.update_one(
        {'_id': summer_axie['_id']},
        {'$set': {'breedTx': breed_tx}}
    )

In [ ]:
from IPython.display import Audio
sound_file = './sound/mixkit-arcade-bonus-alert-767.wav'

Audio(sound_file, autoplay=True)

# Usage

In [ ]:
db.summeraxies.find_one()

In [ ]:
import pandas as pd

cursor = db.summeraxies.find()
summer_axies = pd.DataFrame()
for tx in cursor:
    if tx['breedTx']==None:
        continue
    axie_attributes = {
        'axieId': tx['id'],
        'numSummer': tx['numSummer'],
        'numShiny': tx['numShiny'],
        'createdAt': tx['breedTx']['createdAt'],
        'createDate': tx['breedTx']['createdAt'].date(),
        'txHash': tx['breedTx']['hash'],
        'breeder': tx['breedTx']['from'],
        'sireId': tx['breedTx']['sireId'],
        'matronId': tx['breedTx']['matronId'],
        'feeAxs': tx['breedTx']['feeAxs'],
        'feeSlp': tx['breedTx']['feeSlp']
    }
    for part in tx['parts']:
        if part['type'] == 'Eyes':
            axie_attributes['part_eyes'] = part['id']
        elif part['type'] == 'Ears':
            axie_attributes['part_eyes'] = part['id']
        elif part['type'] == 'Back':
            axie_attributes['part_eyes'] = part['id']
        elif part['type'] == 'Mouth':
            axie_attributes['part_eyes'] = part['id']
        elif part['type'] == 'Horn':
            axie_attributes['part_eyes'] = part['id']
        elif part['type'] == 'Tail':
            axie_attributes['part_eyes'] = part['id']
    summer_axies = pd.concat(
        [summer_axies, pd.DataFrame(axie_attributes, index=[0])],
        ignore_index=True
    )
summer_axies

In [ ]:
usage_by_date = summer_axies.groupby(by='createDate').count()\
    .reset_index()\
    .filter(['createDate', 'axieId'])\
    .rename(columns={'axieId': 'count'})
usage_by_date

In [ ]:
import plotly.graph_objects as go

fig = go.Figure(go.Scatter(x=usage_by_date['createDate'], y=usage_by_date['count']))
fig.update_layout(
    template='plotly_white',
    width=400, height=300,
    margin={'l':10, 'r':10, 't':60, 'b':10},
    yaxis_title='count',
    title='Summer axies bred (only successful breeds)'
)
fig.show()

In [ ]:
breeder_count = summer_axies.groupby(by='breeder').count()\
    .reset_index()\
    .filter(['breeder', 'numSummer'])\
    .rename(columns={'numSummer': 'count'})
breeder_count

In [ ]:
import numpy as np

hist, bin_edges = np.histogram(
    breeder_count['count'], 
    bins=np.logspace(np.log10(1), np.log10(breeder_count['count'].max()),40)
)
fig = go.Figure(go.Scatter(x=bin_edges, y=hist, mode='markers'))
fig.update_layout(
    template='plotly_white', width=400, height=400,
    margin={'l':10, 'r':10, 't':60, 'b':10},
    xaxis={'title': 'number of breeds', 'type': 'log'},
    yaxis={'title': 'number of breeders', 'type': 'log'},
    title='Breeds per breeder'
)
fig.show()

In [ ]:
usage_user_days = summer_axies.groupby(by=['createDate', 'breeder']).count()\
    .reset_index()\
    .filter(['createDate', 'breeder', 'numSummer'])\
    .rename(columns={'numSummer': 'count'})
usage_user_days

In [ ]:
usage_user_days
import numpy as np

hist, bin_edges = np.histogram(
    usage_user_days['count'], 
    bins=np.logspace(np.log10(1), np.log10(usage_user_days['count'].max()), 40)
)
fig = go.Figure(go.Scatter(x=bin_edges, y=hist, mode='markers'))
fig.update_layout(
    template='plotly_white', width=400, height=400,
    margin={'l':10, 'r':10, 't':60, 'b':10},
    xaxis={'title': 'number of breeds', 'type': 'log'},
    yaxis={'title': 'number of days', 'type': 'log'},
    title='Breeds per day (per breeder)'
)
fig.show()

In [ ]:
usage_days = usage_user_days.groupby('breeder').count()\
    .reset_index()\
    .drop('createDate', axis=1)
usage_days

In [ ]:
hist, bin_edges = np.histogram(
    usage_days['count'], 
    bins=np.logspace(np.log10(1), np.log10(usage_days['count'].max()), 40)
)
fig = go.Figure(go.Scatter(x=bin_edges, y=hist, mode='markers'))
fig.update_layout(
    template='plotly_white', width=400, height=400,
    margin={'l':10, 'r':10, 't':60, 'b':10},
    xaxis={'title': 'number of days', 'type': 'log'},
    yaxis={'title': 'number of breeders', 'type': 'log'},
    title='Breeding days (per breeder)'
)
fig.show()

hist = hist / sum(hist)
fig = go.Figure(go.Scatter(x=bin_edges, y=hist, mode='markers'))
fig.update_layout(
    template='plotly_white', width=400, height=400,
    margin={'l':10, 'r':10, 't':60, 'b':10},
    xaxis={'title': 'number of days', 'type': 'log'},
    yaxis={'title': 'P(days)', 'type': 'log'},
    title='Breeding days (per breeder)'
)
fig.show()

In [ ]:
import requests
import time

summer_dates = summer_axies['createDate'].sort_values().unique()

axs_prices = []
slp_prices = []
for date in summer_dates:
    url_axs = "https://api.coingecko.com/api/v3/coins/axie-infinity/history?date=" + date.strftime("%d-%m-%Y")
    r = requests.get(url_axs)
    axs_usd = r.json()['market_data']['current_price']['usd']
    axs_prices += [axs_usd]
    url_slp = "https://api.coingecko.com/api/v3/coins/smooth-love-potion/history?date=" + date.strftime("%d-%m-%Y")
    r = requests.get(url_slp)
    slp_usd = r.json()['market_data']['current_price']['usd']
    slp_prices += [slp_usd]
    time.sleep(5)
prices = pd.DataFrame({'date':summer_dates, 'AXS':axs_prices, 'SLP':slp_prices})
prices

In [ ]:
# prices.to_csv('prices_summer_axs_slp.csv')

In [ ]:
# prices = pd.read_csv('prices_summer_axs_slp.csv')

In [ ]:
import datetime as dt

fees_daily = summer_axies\
    .filter(['createDate', 'feeAxs', 'feeSlp'])\
    .groupby('createDate').sum()\
    .reset_index()
fees_daily['feeAxsInUsd'] = fees_daily['feeAxs'] * prices['AXS']
fees_daily['feeSlpInUsd'] = fees_daily['feeSlp'] * prices['SLP']
fees_daily = fees_daily[fees_daily['createDate'] > dt.date(2022, 7, 10)]
fees_daily

In [ ]:
fig = go.Figure(
    go.Scatter(
        x=fees_daily['createDate'], y=fees_daily['feeAxsInUsd'] + fees_daily['feeSlpInUsd']
    )
)
fig.update_layout(
    template='plotly_white', width=400, height=300,
    margin={'l':10, 'r':10, 't':60, 'b':10},
    yaxis={'title': 'number of axies burned'},
    title='Fees combined (Lunacian Summer)'
)
fig.show()

In [ ]:
import datetime as dt
from plotly.subplots import make_subplots

fig = make_subplots(specs=[[{"secondary_y": True}]])
fig.add_trace(go.Scatter(x=fees_daily['createDate'], y=fees_daily['feeAxsInUsd'], name='AXS fees'))
fig.add_trace(go.Scatter(
    x=fees_daily['createDate'], y=fees_daily['feeSlpInUsd'], name='SLP fees'), secondary_y=True)
fig.update_layout(
    template='plotly_white', width=480, height=300,
    margin={'l':10, 'r':10, 't':60, 'b':10},
    yaxis={'title': 'value burned $'},#, 'type': 'log'},
    title='Fees (Lunacian Summer)'
)
# fig.update_yaxes(type='log', secondary_y=True)
fig.show()

fig = go.Figure(
    go.Scatter(
        x=fees_daily['createDate'], y=fees_daily['feeAxsInUsd'] + fees_daily['feeSlpInUsd']
    )
)
fig.update_layout(
    template='plotly_white', width=400, height=300,
    margin={'l':10, 'r':10, 't':60, 'b':10},
    yaxis={'title': 'number of axies burned'},
    title='Fees combined (Lunacian Summer)'
)
fig.show()

In [ ]:
total_axs_sunk = fees_daily['feeAxs'].sum()
total_slp_sunk = fees_daily['feeSlp'].sum()
total_axs_usd_sunk = fees_daily['feeAxsInUsd'].sum()
total_slp_usd_sunk = fees_daily['feeSlpInUsd'].sum()
total_usd_sunk = total_axs_usd_sunk + total_slp_usd_sunk
print(f"{total_axs_sunk:,.1f} AXS = ${total_axs_usd_sunk:,.2f}")
print(f"{total_slp_sunk:,d} SLP = ${total_slp_usd_sunk:,.2f}")
print(f"\t= ${total_usd_sunk:,.2f}")

In [ ]:
import math

slp_usage = summer_axies\
    .filter(['createDate', 'feeSlp'])\
    .groupby('createDate').mean().reset_index()
slp_usage['count'] = summer_axies\
    .filter(['createDate', 'feeSlp'])\
    .groupby('createDate').count().reset_index()['feeSlp']
slp_usage['std'] = summer_axies\
    .filter(['createDate', 'feeSlp'])\
    .groupby('createDate').std().reset_index()['feeSlp']
slp_usage['stderr'] = slp_usage['std'] / np.sqrt(slp_usage['count'].values)

fig = go.Figure()
fig.add_trace(go.Scatter(x=slp_usage['createDate'], y=slp_usage['feeSlp']))
# fig.add_trace(go.Scatter(
#     x=slp_usage['createDate'].append(slp_usage['createDate'].sort_values(ascending=False)),
#     y=(slp_usage['feeSlp'] + slp_usage['stderr']).append(slp_usage['feeSlp'] - slp_usage['stderr']),
#     fill='tozerox'
# ))
fig.update_layout(
    template='plotly_white', width=500, height=400,
    margin={'l':10, 'r':10, 't':60, 'b':10},
    yaxis={'title': 'SLP', 'range': [0, slp_usage['feeSlp'].max()]},
    title='Average SLP used per breed (Lunacian Summer)'
)
fig.show()

In [ ]:
breeds = summer_axies\
    .filter(['numSummer', 'createDate', 'breeder'])\
    .groupby(['breeder', 'numSummer']).count()\
    .reset_index()\
    .rename(columns={'createDate': 'count'})
breeds.sort_values(by='count', ascending=False)

2022/08/26

Thoughts
- Goal for design is to make slope flatter
    - So reward behavior at tails.
    - Compare the two events. Slopes.
- LS is a feedback loop. Illustration.
    - getting reward & reward being liquid (market, LE need to claim)
- LE tanks the market. LS is slower.
    - length of breeding event should have been shorter. - Kyroh
        - breeders selling to breeders?
        - point out in process

---

To-do
- [ ] Get all breed attempts
- [ ] How much money did people spend? versus event participation? opportunity cost?
- [ ] Secondary sales
- [ ] Profitable in the beginning versus end. Did they get greedy?
    - participation = f(EV)
- [ ] Who is selling versus holding
    - how does distribution change over time, per person
    - basal archetype
        - trader, breeder: sell at end
        - collector: buy at end
        - clustering of archetypes
    - [ ] Do the initial rolls affect behavior?
- [ ] model the demand curve?
- [ ] what percentage of breeds were lunacian summer breeds?

---

Set up
- [ ] Google Doc for paper
- [ ] Notion for notes
  
---------
  
- LTV mining versus retention
- Kitty Inu, no community, UA + retention

### All breeds...

In [ ]:
db.transactions.find_one()

In [ ]:
import datetime as dt

db.transactions.find_one({
    'to': contract_address_axie.lower(),
    'created_at': {'$gt': dt.datetime(2022, 7, 14, 0, 0, 0)}
})

In [ ]:
from web3 import Web3, contract
import json, requests

USER_AGENT = "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.77 Safari/537.36"
headers = {
  "Content-Type": "application/json",
  "User-Agent": USER_AGENT }

web3 = Web3(Web3.HTTPProvider('https://api.roninchain.com/rpc', request_kwargs={ "headers": headers }))

In [ ]:
contract_address_axie = Web3.toChecksumAddress('0x32950db2a7164ae833121501c797d79e7b79d74c')
contract_address_axs = Web3.toChecksumAddress('0x97a9107c1793bc407d6f527b77e7fff4d812bece')
contract_address_slp = Web3.toChecksumAddress('0xa8754b9fa15fc18bb59458815510e40a12cd2014')
contract_address_axie, contract_address_axs, contract_address_slp

In [ ]:
f = open("node_modules/ronin-tx-decoder/abi/axieContract.abi", "r")
lines = f.read()
abi_str = json.dumps(json.loads(lines))
contract_axie = web3.eth.contract(address=contract_address_axie, abi=abi_str)

In [ ]:
USER_AGENT = "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.77 Safari/537.36"
headers = {
  "Content-Type": "application/json",
  "User-Agent": USER_AGENT }
url = 'https://graphql-gateway.axieinfinity.com/graphql'
null = None
auction_types = ["All", "Sale"]
axie_classes = [
    'Beast', 'Aquatic', 'Plant', 'Bug', 'Bird', 'Reptile', 'Mech', 'Dawn', 'Dusk']

def getAxieDetail(axie_id):
    body = {
        "operationName": "GetAxieDetail",
        "variables": {
            "axieId": axie_id,
        },
#         "query": "query GetAxieDetail($axieId: ID!) {\n  axie(axieId: $axieId) {\n    ...AxieDetail\n    __typename\n  }\n}\n\nfragment AxieDetail on Axie {\n  id\n  image\n  class\n  chain\n  name\n  genes\n  newGenes\n  owner\n  birthDate\n  bodyShape\n  class\n  sireId\n  sireClass\n  matronId\n  matronClass\n  stage\n  title\n  breedCount\n  level\n  figure {\n    atlas\n    model\n    image\n    __typename\n  }\n  parts {\n    ...AxiePart\n    __typename\n  }\n  stats {\n    ...AxieStats\n    __typename\n  }\n  order {\n    ...OrderInfo\n    __typename\n  }\n  ownerProfile {\n    name\n    __typename\n  }\n  battleInfo {\n    ...AxieBattleInfo\n    __typename\n  }\n  children {\n    id\n    name\n    class\n    image\n    title\n    stage\n    __typename\n  }\n  potentialPoints {\n    beast\n    aquatic\n    plant\n    bug\n    bird\n    reptile\n    mech\n    dawn\n    dusk\n    __typename\n  }\n  __typename\n}\n\nfragment AxieBattleInfo on AxieBattleInfo {\n  banned\n  banUntil\n  level\n  __typename\n}\n\nfragment AxiePart on AxiePart {\n  id\n  name\n  class\n  type\n  specialGenes\n  stage\n  abilities {\n    ...AxieCardAbility\n    __typename\n  }\n  __typename\n}\n\nfragment AxieCardAbility on AxieCardAbility {\n  id\n  name\n  attack\n  defense\n  energy\n  description\n  backgroundUrl\n  effectIconUrl\n  __typename\n}\n\nfragment AxieStats on AxieStats {\n  hp\n  speed\n  skill\n  morale\n  __typename\n}\n\nfragment OrderInfo on Order {\n  id\n  maker\n  kind\n  assets {\n    ...AssetInfo\n    __typename\n  }\n  expiredAt\n  paymentToken\n  startedAt\n  basePrice\n  endedAt\n  endedPrice\n  expectedState\n  nonce\n  marketFeePercentage\n  signature\n  hash\n  duration\n  timeLeft\n  currentPrice\n  suggestedPrice\n  currentPriceUsd\n  __typename\n}\n\nfragment AssetInfo on Asset {\n  erc\n  address\n  id\n  quantity\n  orderId\n  __typename\n}\n"
        "query": "query GetAxieDetail($axieId: ID!) {\n  axie(axieId: $axieId) {\n    ...AxieDetail\n    __typename\n  }\n}\n\nfragment AxieDetail on Axie {\n class\n  genes\n  newGenes\n  owner\n  bodyShape\n  class\n  sireId\n  matronId\n parts {\n    ...AxiePart\n    __typename\n  }\n   __typename\n}\n\nfragment AxiePart on AxiePart {\n  id\n  name\n  class\n  type\n  specialGenes\n  stage\n  __typename\n}\n\n"
    }
    response = requests.post(url=url, headers=headers, json=body)
    data = json.loads(response.content)
    return data['data']['axie']

def getAxieParts(axie):
    parts = {}
    for part in axie['parts']:
        parts[part['type']] = part['id']
    return parts

# https://axie.substack.com/p/summerskins
# shiny skins have '-shiny' at the end
summer_parts = {
    'Horn': [
        'horn-watermelon-ice-cream',
        'horn-strawberry-ice-cream',
        'horn-vanilla-ice-cream'
    ],
    'Back': [
        'back-turtle-buoy',
        'back-beach-ball',
    ],
    'Eyes': [
        'eyes-flower-sunglasses',
    ],
    'Mouth': [
        'mouth-bubble-fish',
    ],
    'Tail': [
        'tail-cotton-candy',
    ],
    'Ears': [
        'ears-coca',
    ]
}

def isSummerAttempt(parts_sire, parts_matron):
    # if sire or matron has summer part, is summer attempt
    for key, parts in summer_parts.items():
        if parts_sire[key] in parts:
            return True
        if parts_matron[key] in parts:
            return True
    # check individual combinations
    if ('watermelon' in parts_sire['Horn'] and 'unko' in parts_matron['Horn']) or\
        ('watermelon' in parts_matron['Horn'] and 'unko' in parts_sire['Horn']):
        return True
    if ('strawberry-shortcake' in parts_sire['Horn'] and 'unko' in parts_matron['Horn']) or\
        ('strawberry-shortcake' in parts_matron['Horn'] and 'unko' in parts_sire['Horn']):
        return True
    if ('eggshell' in parts_sire['Horn'] and 'unko' in parts_matron['Horn']) or\
        ('eggshell' in parts_matron['Horn'] and 'unko' in parts_sire['Horn']):
        return True
    if ('balloon' in parts_sire['Horn'] and 'red-ear' in parts_matron['Back']) or\
        ('balloon' in parts_matron['Horn'] and 'red-ear' in parts_sire['Back']):
        return True
    if ('balloon' in parts_sire['Back'] and 'furball' in parts_matron['Back']) or\
        ('balloon' in parts_matron['Back'] and 'furball' in parts_sire['Back']):
        return True
    if ('blossom' in parts_sire['Eyes'] and 'neo' in parts_matron['Eyes']) or\
        ('blossom' in parts_sire['Eyes'] and 'neo' in parts_matron['Eyes']):
        return True
    if ('silence-whisper' in parts_sire['Mouth'] and 'risky-fish' in parts_matron['Mouth']) or\
        ('silence-whisper' in parts_sire['Mouth'] and 'risky-fish' in parts_matron['Mouth']):
        return True
    if ('cloud' in parts_sire['Tail'] and 'cottontail' in parts_matron['Tail']) or\
        ('cloud' in parts_matron['Tail'] and 'cottontail' in parts_sire['Tail']):
        return True
    if ('seaslug' in parts_sire['Ears'] and 'nyan' in parts_matron['Ears']) or\
        ('seaslug' in parts_matron['Ears'] and 'nyan' in parts_sire['Ears']):
        return True
    return False
    

In [ ]:
receipt = web3.eth.get_transaction_receipt(
    '0x033f3bc3dc4d42c50674313bd0dbb58639063834f6b7370f19e6670ff137abd1')
axie_spawn = contract_axie.events.AxieSpawn().processReceipt(receipt)
axie_breed = contract
axie_spawn

In [ ]:
tx_whole = web3.eth.get_transaction(
    '0x033f3bc3dc4d42c50674313bd0dbb58639063834f6b7370f19e6670ff137abd1')
tx_input = contract_axie.decode_function_input(tx_whole.input)
tx_input

In [ ]:
tx_input[1]['_sireId']

In [ ]:
axie = getAxieDetail(2333356)
parts = getAxieParts(axie)
parts, axie['sireId']

In [ ]:
sire_id, matron_id = tx_input[1]['_sireId'], tx_input[1]['_matronId']
sire = getAxieDetail(sire_id)
matron = getAxieDetail(matron_id)
parts_sire = getAxieParts(sire)
parts_matron = getAxieParts(matron)
isSummerAttempt(parts_sire, parts_matron)

In [ ]:
getAxieParts(getAxieDetail(sire_id)), getAxieParts(getAxieDetail(matron_id))

In [ ]:
# testing
summer1 = '0xcd8649f564f5dee7f52c61c22934ad1929144c072bebd6922b19d897196a63ba'
summer2 = '0x033f3bc3dc4d42c50674313bd0dbb58639063834f6b7370f19e6670ff137abd1'
summer3 = '0x9b2fe15da5ac8b60692723874a4c91613777a9b9a7c09ca48d80e34644473c57'
non1 = '0xc4618f712fad8efa13c25ec62f07e10169a314cd69e714b6611fbb3d52e72b82'
non2 = '0x2e06516533a9709f0dcceab6676a69f8066aab665ca9d7ea36c1c316eb6b6f76'
non3 = '0x17bc714988e44ef2262a9ab0368ac52fd821555c43646fe6a2486f5d308f0289'

for tx in [summer1, summer2, summer3, non1, non2, non3]:
    tx_whole = web3.eth.get_transaction(tx)
    tx_input = contract_axie.decode_function_input(tx_whole.input)
    sire_id, matron_id = tx_input[1]['_sireId'], tx_input[1]['_matronId']
    sire = getAxieDetail(sire_id)
    matron = getAxieDetail(matron_id)
    parts_sire = getAxieParts(sire)
    parts_matron = getAxieParts(matron)
    print(isSummerAttempt(parts_sire, parts_matron))

In [ ]:
tx_hash = '0xe31626e039473d16669104bab2ac84a741299aea7494b4e5cd7e32020cc03793'
%timeit web3.eth.get_transaction(tx_hash)
%timeit getAxieDetail(sire_id), getAxieDetail(matron_id)
%timeit web3.eth.get_transaction_receipt(tx['hash'])

In [ ]:
import os
os.system("printf '\a'") # or '\7'

In [ ]:
from IPython.display import Audio
sound_file = './sound/mixkit-arcade-bonus-alert-767.wav'

Audio(sound_file, autoplay=True)

In [ ]:
# using Max's axie dump...

import os
import json

path = '/Users/harang-mbp-22/Developer/data/axie dump'
files = [file for file in os.listdir(path) if file[0] != '.']
maxies = []
for i, file in enumerate(files):
    print(f"File '{file}' {i+1}/{len(files)}.")
    with open(os.path.join(path, file), 'r') as f:
        for line in f:
            data = json.loads(line)
            if '__typename' not in data['Item']:
                continue
            body_shape = data['Item']['bodyShape']['S'] if 'S' in data['Item']['class'] else None
            axie_class = data['Item']['class']['S'] if 'S' in data['Item']['class'] else None
            axie = {
                'axieId': data['Item']['story_id']['S'],
                'matronId': int(data['Item']['matronId']['N']),
                'sireId': int(data['Item']['sireId']['N']),
                'genes': data['Item']['genes']['S'],
                'parts': {part['M']['type']['S']: part['M']['id']['S'] for part in data['Item']['parts']['L']},
                'bodyShape': body_shape,
                'class': axie_class,
            }
            db.maxies.insert_one(axie)

In [ ]:
db.maxies.count_documents({})

In [ ]:
db.maxies.find_one({'axieId': '6994392'}), db.maxies.find_one({'axieId': 'test'})

In [ ]:
db.maxies.create_index([('axieId', pm.DESCENDING)])

In [ ]:
def getAxiePartsDB(axie_id):
    axie = db.maxies.find_one({'axieId': str(axie_id)})
    return axie['parts'] if axie else None

In [ ]:
getAxiePartsDB(6994392), getAxiePartsDB('hi')

In [ ]:
db.transactions.create_index([('created_at', pm.ASCENDING), ('to', pm.TEXT)])
Audio(sound_file, autoplay=True)

In [ ]:
f = open("axie-abi.json", "r")
lines = f.read()
abi_str = json.dumps(json.loads(lines))
contract_axie = web3.eth.contract(address=contract_address_axie, abi=abi_str)
function_breed = contract_axie.find_functions_by_name('breedAxies')[0]

In [ ]:
event_start = dt.datetime(2022, 7, 14, 4, 0, 0)
query = {
    'to': contract_address_axie.lower(),
    'created_at': {'$gt': event_start}
}
query_count = db.transactions.count_documents(query)
cursor = db.transactions.find(query)
print(f"Total txs to query: {query_count}.")

In [ ]:
for i, tx in enumerate(cursor):
    if i%100==0:
        print(f"\r{i}/{query_count}", end='')
    if 'input' in tx:
        continue
    tx_whole = web3.eth.get_transaction(tx['hash']) # ronin call
    db.transactions.update_one(
        {'_id': tx['_id']},
        {
            '$set': {
                'input': tx_whole['input']
            }
        }
    )

Audio(sound_file, autoplay=True)

In [ ]:
import datetime as dt

event_start = dt.datetime(2022, 7, 14, 4, 0, 0)
query = {
    'to': contract_address_axie.lower(),
    'created_at': {'$gt': event_start}
}
query_count = db.transactions.count_documents(query)
cursor = db.transactions.find(query)
print(f"Total txs to query: {query_count}.")

In [ ]:
tx['hash']

In [ ]:
for i, tx in enumerate(cursor):
    if i%100==0:
        print(f"\r{i}/{query_count}", end='')
    if 'input' not in tx:
        continue
    if 'function' in tx:
        continue
    try:
        tx_input = contract_axie.decode_function_input(tx['input'])
    except:
        continue
    if str(tx_input[0]) != str(function_breed):
        continue
    sire_id, matron_id = tx_input[1]['_sireId'], tx_input[1]['_matronId']
    receipt = web3.eth.get_transaction_receipt(tx['hash']) # ronin call
    axie_spawn = contract_axie.events.AxieSpawn().processReceipt(receipt)
    if len(axie_spawn) == 0:
        continue
    fee_axs = fee_slp = 0
    axie_id = axie_spawn[0]['args']['_axieId']
    for log in receipt.logs:
        if log.address==contract_address_axs:
            fee_axs = int(log.data, 16) / 10**18
        elif log.address==contract_address_slp:
            fee_slp = int(log.data, 16)
    db.transactions.update_one(
        {'_id': tx['_id']},
        {
            '$set': {
                'function': 'breedAxies',
                'axieId': axie_id,
                'sireId': sire_id,
                'matronId': matron_id,
                'feeAxs': fee_axs,
                'feeSlp': fee_slp
            }
        }
    )

Audio(sound_file, autoplay=True)

In [ ]:
event_start = dt.datetime(2022, 7, 14, 4, 0, 0)
query = {
    'to': contract_address_axie.lower(),
    'created_at': {'$gt': event_start}
}
query_count = db.transactions.count_documents(query)
cursor = db.transactions.find(query)
print(f"Total txs to query: {query_count}.")

In [ ]:
for i, tx in enumerate(cursor):
    if i%100==0:
        print(f"\r{i}/{query_count}", end='')
    if 'function' not in tx or tx['function'] != 'breedAxies':
        continue
    sire_id = tx['sireId']
    matron_id = tx['matronId']
    sire_parts, matron_parts = getAxiePartsDB(sire_id), getAxiePartsDB(matron_id)
    if not sire_parts:
        sire_parts = getAxieParts(getAxieDetail(sire_id)) # graphql call
    if not matron_parts:
        matron_parts = getAxieParts(getAxieDetail(matron_id)) # graphql call
    summer_attempt = isSummerAttempt(sire_parts, matron_parts)
    db.transactions.update_one(
        {'_id': tx['_id']},
        {
            '$set': {
                'summerAttempt': True
            }
        }
    )

Audio(sound_file, autoplay=True)

In [ ]:
# parts

In [ ]:
# network of summer axies?

### Market

- how much
- when
- n/6?
    - then breed?
- do markets value rarity?